# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_imdb(title_type, user_rating, start_date, end_date):
    base_url = "https://www.imdb.com/search/title/"
    params = {
        'title_type': title_type,
        'release_date': f"{start_date},{end_date}",
        'user_rating': f"{user_rating},",
        'start': 1
    }
    
    movie_data = []
    while True:
        response = requests.get(base_url, params=params)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        movies = soup.find_all('div', class_='lister-item mode-advanced')
        if not movies:
            break
        
        for movie in movies:
            movie_nr = movie.find('span', class_='lister-item-index').text.strip('.')
            title = movie.h3.a.text
            year = movie.h3.find('span', class_='lister-item-year').text.strip('()')
            rating = movie.find('div', class_='inline-block ratings-imdb-rating')['data-value']
            runtime = movie.find('span', class_='runtime').text.strip(' min') if movie.find('span', class_='runtime') else None
            genre = movie.find('span', class_='genre').text.strip() if movie.find('span', class_='genre') else None
            description = movie.find_all('p', class_='text-muted')[1].text.strip()
            director = movie.find('p', class_='').find_all('a')[0].text
            stars = ', '.join([a.text for a in movie.find('p', class_='').find_all('a')[1:]])
            votes = movie.find('span', attrs={'name': 'nv'})['data-value']
            gross = movie.find_all('span', attrs={'name': 'nv'})[1].text.strip('$M') if len(movie.find_all('span', attrs={'name': 'nv'})) > 1 else None
            
            movie_data.append({
                'Movie Nr': movie_nr,
                'Title': title,
                'Year': year,
                'Rating': rating,
                'Runtime (min)': runtime,
                'Genre': genre,
                'Description': description,
                'Director': director,
                'Stars': stars,
                'Votes': votes,
                'Gross ($M)': gross
            })
        
        next_page = soup.find('a', class_='lister-page-next next-page')
        if next_page:
            params['start'] += 50
        else:
            break
    
    return pd.DataFrame(movie_data)

# Example usage
df = scrape_imdb('feature', 7.5, '1990-01-01', '1992-12-31')
print(df)